### Measurement Process - continuous experiment: regression background
relevant part of json schema:<br>
```json
"characteristicData": [
    "values":[
        {"level": "A", "part": 1, "repetition": 1, "value": 2.85},
        ..etc        
    ],
    "numberOfLevels": 2,
    "numberOfRepetitions": 4,
    "numberOfParts": 3,
    etc..

]
```
Produces a table:<br>
<table>
    <tr>
        <th>  </th>
        <th>part 1</th>
        <th>part 2</th>
        <th>part 3</th>
    </tr>
    <tr>
        <td>level A</td>
        <td>..</td>
        <td>..</td>
        <td>..</td>
    </tr>
    <tr>
        <td>level B</td>
        <td>..</td>
        <td>..</td>
        <td>..</td>        
    </tr>
</table>
Where in each cell can be a "numberOfRepetitions" of values (balanced case). However it can happen that some cells lack a number (unbalanced case).<br>
__Goals__: test for the interactions of row (levels) and column (parts) effects, and report the portion of variance due to interactions (if they are significant), portion of variance due to error and due to the levels effect. 

Use Multiple Linear regression with categorical variables. Interactions are represented with "cross/mixed products". For the example of the table above, we have variables <br>$x_b = 1$ if is level B, 0 otherwise,<br> $t_2 = 1$ if is part 2, 0 otherwise,<br> $t_3=1$ if is part 3, 0 otherwise<br>
and construct a full model like so:
$$\beta_0 + \beta_1 x_b + \beta_2 t_2 + \beta_3 t_3 + \beta_4 x_b t_2 + \beta_5 x_b t_3$$

Call <font face='Consolas'>numpy.linalg.lstsq</font> to get the $\beta$ (<font face='Consolas'>beta_full</font>) and sum of squares of residuals (ie: measured - fitted). Call this sum of squares <font face='Consolas'>residuals_full</font>, the associated degrees of freedom <font face='Consolas'>df_full = len(all_values) - len(beta_full)</font>. If we refer to the traditional ANOVA table (ISO standard page 48, table B.3.4.), the <font face='Consolas'>residuals_full</font> is actually $SS_R$.

__Testing for interactions__: The null hypothesis is that interactions are not significant, in our example that means: $H_0: \beta_4 = \beta_5 = 0$ vs at least on of them is not zero. Hence we have the reduced model: 
$$\beta_0 + \beta_1 x_b + \beta_2 t_2 + \beta_3 t_3$$

Fit the reduced model to get ```beta_reduced_inter, residuals_reduced_inter```.We will use the F-test [https://en.wikipedia.org/wiki/F-test] (scroll down to Regression problems) to compare the full and the reduced model. So we have  
```MS_interactions = (residuals_reduced_inter - residuals_full) / (df_reduced_inter - df_full)```

and ```MS_full = residuals_full / df_full```

and the F-statistic is ```F_interactions = MS_interactions / MS_full``` realization of the F random variable with (<font face='Consolas'>df_reduced_inter - df_full, df_full</font>) degrees of freedom.<br>

__Estimate of variance due to interactions__: The null hypothesis is rejected if the F calculated from the data is greater than the critical value. In that case, we can look for the variance due to interactions. In the standard ANOVA, this is estimated as ```(MS_interactions - MS_full) / numberOfRepetitions``` (refer to table B.3.5. in ISO page 48). But in the unbalanced case, the number of repetitions is not always the same. Notice that the in the balanced case, the total number of all measured values is <br>
```numberOfParts * numberOfLevels * numberOfRepetitions = len(all_values)``` <br>
hence <br>
```numberOfRepetitions = len(all_values) / numberOfParts * numberOfLevels```


call the right-hand side a <font face='Consolas'>correction</font>. Now the estimate of variance due to interactions is ```(MS_interactions - MS_full) / correction```

__Estimate of variance due to row factor__: Fit a reduced model (following the example above): $\beta_0 + \beta_2 t_2 + \beta_3 t_3 $ to get ```beta_reduced_levels, residuals_reduced_levels```. Referring to ISO page 48, table B.3.5, the estimate is calculated as <font face='Consolas'>(MS_levels - MS_interactions) / numberOfParts * numberOfRepetitions</font>. Where <br>```MS_levels = (residuals_reduced_levels - residuals_reduced_inter) / (df_reduced_levels - df_reduced_inter)```<br>

and the F-statistic is ```MS_levels / MS_full``` a realization of the F random variable with ```(df_reduced_levels - df_full, df_full)``` degrees of freedom.

(notice the comparison of the two reduced models in nominator of MS, <font face='Consolas'>reduced_levels</font> is immediately nested in <font face='Consolas'>reduced_inter</font> and <font face='Consolas'>reduced_inter</font> is immediately nested in <font face='Consolas'>full</font>)<br>

So the estimate of variance due to levels factor (if interactions are significant) becomes ```(MS_levels - MS_interactions) / (correction * numberOfParts)```. 

If they are not significant, construct  ```SS_pool = SS_full + SS_inter``` and the estimate is ```(MS_levels - MS_pool) / (correction * numberOfParts)``` 

__Estimate of variance due to error__: is simply ```MS_full```.

### Example - comparison with standard ANOVA

In [61]:
import pandas as pd
import numpy as np
from scipy.stats import f
from math import sqrt
import json, string

In [62]:
 def regression(char_data, numberOfRepetitions, numberOfParts, numberOfLevels):
    # output table
    out = pd.DataFrame()
    
    # get all values
    all_values = []
    unbalanced = False
    for row in table.index:
        for col in table.columns:
            all_values.append(table.loc[row, col])
            if len(table.loc[row, col]) != numberOfRepetitions:
                unbalanced = True

    all_values = [item for sublist in all_values for item in sublist]

    # make design matrix for regression with dummy variables
    ones = len(all_values) * [1]
    
    # dummy variables for columns (parts)
    parts = []
    for col_main in char_data.columns[1:]:    
        is_part = []
        for row in char_data.index:
            for col in char_data.columns:
                if col == col_main:
                    is_part.append([1] * len(char_data.loc[row,col]))
                else:
                    is_part.append([0] * len(char_data.loc[row,col]))

        is_part = [item for sublist in is_part for item in sublist]
        parts.append(is_part)
    
    # dummy variables for rows (levels)
    levels = []
    for row_main in char_data.index[1:]:
        is_level = []
        for row in char_data.index:
            for col in char_data.columns:
                if row == row_main:
                    is_level.append([1] * len(char_data.loc[row,col]))
                else:
                    is_level.append([0] * len(char_data.loc[row,col]))

        is_level = [item for sublist in is_level for item in sublist]
        levels.append(is_level)  
    
    # dummy variables for interaction terms
    mixed = []
    for level in levels:
        for part in parts:
            mixed.append(np.array(level)*np.array(part))

    X_full = np.vstack((ones, levels, parts, mixed))
    X_full = np.transpose(X_full)
    beta_full, residuals_full = np.linalg.lstsq(X_full, all_values, rcond=None)[0:2]
    df_full = len(all_values) - len(beta_full)
    MS_full = residuals_full / df_full

    # testing for interactions 
    X_reduced_inter = np.transpose(np.vstack((ones, levels, parts)))
    beta_reduced_inter, residuals_reduced_inter =  np.linalg.lstsq(X_reduced_inter, all_values, rcond=None)[0:2]
    df_reduced_inter = len(all_values) - len(beta_reduced_inter)

    MS_interactions = (residuals_reduced_inter - residuals_full) / (df_reduced_inter - df_full)
    
    F_interactions = MS_interactions / MS_full
    critical_interactions = f.ppf(1-0.05, df_reduced_inter - df_full, df_full)    
    
    
    # testing for row factor
    X_reduced_levels = np.transpose(np.vstack((ones, parts)))
    beta_reduced_levels, residuals_reduced_levels = np.linalg.lstsq(X_reduced_levels, all_values, rcond=None)[0:2]
    df_reduced_levels = len(all_values) - len(beta_reduced_levels)

    MS_levels = (residuals_reduced_levels - residuals_reduced_inter) / (df_reduced_levels - df_reduced_inter) 
    F_levels = MS_levels / MS_full
    critical_levels = f.ppf(1-0.05, df_reduced_levels - df_reduced_inter, df_full)

    
    # testing for column factor 
    X_reduced_columns = np.transpose(np.vstack((ones, levels)))
    beta_reduced_columns, residuals_reduced_columns = np.linalg.lstsq(X_reduced_columns, all_values, rcond=None)[0:2]
    df_reduced_columns = len(all_values) - len(beta_reduced_columns)

    MS_parts = (residuals_reduced_columns - residuals_reduced_inter) / (df_reduced_columns - df_reduced_inter)
    F_parts = MS_parts / MS_full
    critical_columns = f.ppf(1-0.05, df_reduced_columns - df_reduced_inter, df_full)
    
    correction = len(all_values) / (numberOfLevels * numberOfParts)
    
    if F_interactions > critical_interactions:
        # interactions are significant   
        print('interactions are significant')
       
        u_GI = sqrt((MS_levels - MS_interactions) / (correction * numberOfParts))      

        u_IA = sqrt((MS_interactions - MS_full) / correction)

        u_EVO = sqrt(MS_full)
        
    else:
        print('interactions are not significant')
        SS_evo = (MS_full * df_full) 
        SS_IA = MS_interactions * (df_reduced_inter - df_full)
        
        SS_pool = SS_evo + SS_IA
        df_pool = df_reduced_inter
        MS_pool = SS_pool / df_pool
        
        print('SS_evo (SS_within)', SS_evo)
        print('SS_inter', SS_IA)
        print('df_pool', df_pool)
        print('SS_pool = SS_evo + SS_inter', SS_evo + SS_IA)
        print('MS_pool', MS_pool)
        

        u_EVO = sqrt(MS_pool)
        #u_GI = sqrt((MS_levels - MS_pool) / correction * numberOfParts)
        print('MS_levels', MS_levels)
        u_IA = 0
        
        print('u_EVO', u_EVO)
        #print('u_GI', u_GI)
        print('u_IA', u_IA)
        
              
    out = out.append(
        {'A_Df': df_reduced_levels - df_reduced_inter, 
         'B_MS': np.round(MS_levels[0], 4), 
         'F': np.round(F_levels[0],4), 
         'F crit': critical_levels},
        ignore_index=True)
    
    out = out.append(
        {'A_Df': df_reduced_columns - df_reduced_inter, 
         'B_MS': np.round(MS_parts[0], 4),
         'F': np.round(F_parts[0],4),
         'F crit': critical_columns},
        ignore_index=True)
    out = out.append(
        {'A_Df': df_reduced_inter - df_full, 
         'B_MS': np.round(MS_interactions[0], 4),
         'F': np.round(F_interactions[0], 4),
         'F crit': critical_interactions},
        ignore_index=True)
    out = out.append(
        {'A_Df': df_full, 
        'B_MS': np.round(MS_full[0], 4),
        'F': [],
        'F crit': []},
        ignore_index=True)
    out.index = ['Rows', 'Columns', 'Interactions', 'Within']
        
    return out

In [63]:
d = {'part 1': [[2.838, 4.216, 2.889, 4.198], [1.884,2.283,4.939,3.486]], 
     'part 2': [[3.55, 4.556,3.087,1.943],[2.396,2.956,3.105,2.649]],
     'part 3': [[3.620,3.079,3.586,2.669],[2.801,3.421,4.275,3.110]]}
rows = ['level A', 'level B']
table = pd.DataFrame(data=d, index=rows)
table

,part 1,part 2,part 3
level A,"[2.838, 4.216, 2.889, 4.198]","[3.55, 4.556, 3.087, 1.943]","[3.62, 3.079, 3.586, 2.669]"
level B,"[1.884, 2.283, 4.939, 3.486]","[2.396, 2.956, 3.105, 2.649]","[2.801, 3.421, 4.275, 3.11]"


In [64]:
regression(table, 4, 3, 2)

interactions are not significant
SS_evo (SS_within) [13.1263235]
SS_inter [0.51161058]
df_pool 20
SS_pool = SS_evo + SS_inter [13.63793408]
MS_pool [0.6818967]
MS_levels [0.35672817]
u_EVO 0.8257703701191189
u_IA 0


,A_Df,B_MS,F,F crit
Rows,1.0,0.3567,0.4892,4.41387
Columns,2.0,0.2419,0.3318,3.55456
Interactions,2.0,0.2558,0.3508,3.55456
Within,18.0,0.7292,[],[]


$u_{AV} = \sqrt{\frac{MS_{levels} - MS_{pool}}{wn}} $<-- ISO. In this example negative number under root: MS_levels - MS_pool < 0.

Compare this table with Excel.<br>
<img src='excel.jpg'>

In [65]:
# example 2
with open ('example_2.json') as fh:
    values = json.load(fh)
values = values['values']

In [66]:
def get_values(values, numberOfLevels, numberOfParts, numberOfRepetitions):
    """returns a DataFrame table for ANOVA"""
    levels = [x for x in string.ascii_uppercase[0:numberOfLevels]]


    parts = [str(x) for x in range(1,numberOfParts+1)]

    table = pd.DataFrame(index=levels, columns=parts)
    for level in levels:
        for part in parts:
            table.loc[level, part] = []
    
    for row in values:
        table.loc[row['level'], str(row['part'])].append(row['value'])
    return table

In [69]:
numberOfLevels = 3
numberOfParts = 10
numberOfRepetitions = 3
table = get_values(values, numberOfLevels, numberOfParts, numberOfRepetitions)
table

,1,2,3,4,5,6,7,8,9,10
A,"[8.12, 8.435, 8.48]","[7.445, 6.815, 7.49]","[9.965, 10.01, 9.56]","[6.14, 5.96, 6.365]","[5.69, 5.6, 5.78]","[2.855, 2.45, 2.585]","[10.685, 10.595, 10.775]","[6.725, 6.275, 6.545]","[4.97, 5.105, 5.51]","[9.875, 10.1, 9.875]"
B,"[8.2, 8.29, 8.245]","[7.3, 7.12, 7.075]","[9.66, 9.34, 9.25]","[6.095, 6.185, 6.185]","[5.08, 5.34, 5.44]","[2.315, 2.585, 2.315]","[10.45, 10.84, 11.05]","[6.24, 6.12, 6.3]","[5.015, 5.285, 5.15]","[10.08, 9.8, 9.97]"
C,"[8.525, 8.435, 8.345]","[7.535, 7.355, 7.085]","[9.83, 9.695, 9.515]","[6.14, 6.14, 6.05]","[5.78, 5.735, 5.555]","[2.63, 2.36, 2.585]","[10.865, 11.0, 11.18]","[6.59, 6.5, 6.725]","[5.06, 5.195, 5.105]","[10.19, 9.785, 9.965]"


In [68]:
regression(table, numberOfRepetitions, numberOfParts, numberOfLevels)

interactions are not significant
SS_evo (SS_within) [1.91728333]
SS_inter [0.68593389]
df_pool 78
SS_pool = SS_evo + SS_inter [2.60321722]
MS_pool [0.03337458]
MS_levels [0.25953028]
u_EVO 0.18268710893787715
u_IA 0


,A_Df,B_MS,F,F crit
Rows,2.0,0.2595,8.1218,3.15041
Columns,9.0,58.5419,1832.03,2.0401
Interactions,18.0,0.0381,1.1925,1.77845
Within,60.0,0.0320,[],[]


Compare with excel:<br>
<img src="excel_2.jpg">
In this example substracting MS_levels - MS_pool happens to be ok.